In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from scipy import integrate
import sys
sys.path.append('../series_alignment/')
sys.path.append('../MC-DFM/')
from Scattering_Simulator import pairwise_method  #as pairwise_method
import SAXS


In [ ]:
file_path = '../Data/Simulations/Simulation_File_8/Optimization_Results/Sample_0/Density_0.005_U_0_46.95_r0_2.25_n_1.0_m_20.89/DNA_assembly_20250902223201.gsd' #This is the path to the .gsd file 

In [ ]:
positions, orientations = SAXS.extract_positions_orientations(file_path)
print(f"Number of frames: {len(positions)}")
print("First frame position shape:", positions[0].shape)
print("First frame orientation shape:", orientations[0].shape if orientations[0] is not None else "Not available")

In [ ]:
orientations_euler = []
for i in range(len(orientations)):
    angles = SAXS.quaternion_to_euler(orientations[i][:, :]) 
    orientations_euler.append(angles)

lattice_coordinates = []
for i in range(len(orientations_euler)):
    lattice_coordinates.append(np.hstack((positions[i][:, :], orientations_euler[i][:, :])))

In [ ]:
#Sphere
D = 1            # diameter of the sphere
spacing = 0.03    # spacing between grid points
points = SAXS.grid_points_in_sphere(D, spacing)
ones = np.array([1]*len(points))
points = np.hstack((points, ones.reshape(-1,1)))

################################## INPUTS ##########################################################################################
histogram_bins = 10000            # Number of bins in the pairwise distribution calculation 
q = np.geomspace(0.00008, 20, 1000)  # SAXS q-range to be calculated
# Where to save the files and plot
####################################################################################################################################
new_directory = file_path[:-31] + 'scattering_data_mcdfm'
#os.mkdir(new_directory)
frames = len(lattice_coordinates)
cmap = cm.jet
norm = plt.Normalize(0, frames - 1)
plt.rcParams.update({'font.size': 18})
fig, ax = plt.subplots(figsize=(10,7))
for i in range(frames):
    n_samples = 10000000
    simulator = pairwise_method.scattering_simulator(n_samples)
    simulator.sample_building_block(points)
    simulator.sample_lattice_coordinates(lattice_coordinates[i])
    simulator.calculate_structure_coordinates()
    #I_q = simulator.simulate_multiple_scattering_curves_lattice_coords(points, lattice_coordinates[i], histogram_bins, q, save=False).cpu().numpy()
    #I_q = np.mean(I_q, axis=1)
    I_q = simulator.simulate_scattering_curve_fast_lattice(points, lattice_coordinates[i], histogram_bins, q, save=False).cpu().numpy()
    plt.rcParams.update({'font.size': 18})
    q_rescaled = q/260        
    ax.plot(q_rescaled, I_q, color=cmap(norm(i)), linewidth = 3, label = '10m Sample Size')
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylabel('Intensity (arb. unit)')
    ax.set_xlabel('q ($\\AA^{-1}$)')
    #plt.legend(fontsize=14)
    data = np.hstack((q_rescaled.reshape(-1,1), I_q.reshape(-1,1)))
    #np.save(new_directory + '/data_' + str(i) + '.npy', data)   
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([]) 
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Frame')
    #plt.savefig(new_directory + '/scattering_curve_plot.png', dpi=600, bbox_inches="tight")
    plt.close()

In [ ]:
path = new_directory
filenames = sorted(os.listdir(path))
for i in range(len(filenames)):
    if len(filenames[i]) == 10:
        new_name = filenames[i][0:-5] + '0' + filenames[i][-5:]
        os.rename(path + '/' + filenames[i], path + '/' + new_name)

In [ ]:
path = new_directory
filenames = sorted(os.listdir(path))

norm = colors.Normalize(vmin=0, vmax=1)
cmap = cm.jet
sm = cm.ScalarMappable(norm=norm, cmap=cmap)
times = np.linspace(0, 1, len(filenames))
fig, ax = plt.subplots(figsize=(10,7))
for i in range(len(filenames)):
    data = np.load(path + '/' + filenames[i])
    ax.plot(data[:,0], data[:,1], linewidth = 3, color=sm.to_rgba(times[i]))
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylabel('Intensity (arb. unit)')
    ax.set_xlabel('q ($\\AA^{-1}$)')
    ax.set_xlim([0.0001, 0.1])

cbar = fig.colorbar(sm, ax=ax, orientation='vertical')
cbar.set_label('Frame of Simulation (normalized)')
plt.tight_layout()


## Convert Data into Structure Factors

In [ ]:
def sphere(q, r):
    return 3*(np.sin(q*r) - q*r*np.cos(q*r))**2/(q*r)**6
q = np.geomspace(0.003, 0.08, 500)
I_sphere = sphere(q, 130)
monodisperse_sphere = np.hstack((q.reshape(-1,1), I_sphere.reshape(-1,1)))

In [ ]:
file_path = '../Data/Simulations/Simulation_File_8/Optimization_Results/Sample_0/Density_0.005_U_0_46.95_r0_2.25_n_1.0_m_20.89/scattering_data_mcdfm/'
filenames = sorted(os.listdir(file_path))
for i in range(len(filenames)):
    data0 = np.load(file_path + filenames[i])
    s_q = SAXS.calculate_structure_factor(data0, monodisperse_sphere, 0.02, 0.03, False)
    #np.save('Simulation_File_8/Optimization_Results/Sample_0/Density_0.005_U_0_46.95_r0_2.25_n_1.0_m_20.89/scattering_data_mcdfm_sq/' + filenames[i], s_q)

## Convert Experimental Data into Structure Factor

In [ ]:
file_path = '../Data/SAXS/Kinetics/ESAXS_Sub/'
#sphere_polydisperse = read_DAT_file('../Experimental_data/sasmodels_sphere_fit')
filenames = sorted(os.listdir(file_path))
for i in range(len(filenames)):
    data0 = SAXS.read_DAT_file(file_path + filenames[i])
    s_q = SAXS.calculate_structure_factor(data0, monodisperse_sphere, 0.02, 0.03, False)
    #np.save('../Data/SAXS/Kinetics_Sq/' + filenames[i], s_q)